In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
#Install the required packages for this project
!pip install einops datasets bitsandbytes accelerate peft flash_attn
# !pip uninstall -y transformers
# !pip install git+https://github.com/huggingface/transformers
# !pip install --upgrade torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 61.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 38.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-p

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig

model_name = "microsoft/phi-2"

# Configuration to load model in 4-bit quantized
bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type='nf4',
                                bnb_4bit_compute_dtype='float16',
                                #bnb_4bit_compute_dtype=torch.bfloat16,
                                bnb_4bit_use_double_quant=True)


#Loading Microsoft's Phi-2 model with compatible settings
model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto',
                                             quantization_config=bnb_config,
                                             #attn_implementation="flash_attention_2",# to increase the training speed
                                             trust_remote_code=True)

# Setting up the tokenizer for Phi-2
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          add_eos_token=True,
                                          trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.truncation_side = "left"


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
print(f"Memory footprint: {model.get_memory_footprint() / 1e9} GB")

Memory footprint: 1.792884736 GB


In [ ]:
!pip install py2neo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 4.6 MB/s eta 0:00:00


In [ ]:
from py2neo import Graph, Node, Relationship
graph = Graph("bolt+s://d0642942.databases.neo4j.io:7687", auth=("neo4j", "AHHzMMC8_dusNm8T6bdkgs0lClYkdUg3VvChFi8d5GI"))

In [ ]:
import pandas as pd
from datasets import Dataset

# Read the excel file
df = pd.read_excel('/content/drive/MyDrive/ontology/uber.xlsx', header=None)
train_dataset = Dataset.from_pandas(df[1:900])
test_dataset = Dataset.from_pandas(df[900:1114])

In [ ]:
train_dataset[1]

{'0': 'How do I create an Uber account?',
 '1': "Creating an Uber account requires a valid email address and phone number. You'll also need to create a password and agree to terms and conditions and our privacy notice.\n\nFill in your first and last name, phone number, and preferred language. Once you complete this part of the signup process, we send a text SMS to verify your phone number.\n\nNext, enter your payment information. Adding a credit card or debit card number allows your trip fares to be automatically charged after each ride. Please note that we cannot accept prepaid cards.\n\nAfter you provide this info, we'll send an email to confirm your account registration. Once you confirm, you'll be able to use your app to request a ride."}

In [ ]:
import re

def preprocess_text(text):
    try:
      text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
      text = re.sub(r'\W', ' ', text)  # Remove non-alphanumeric characters
      text = text.lower().strip()  # Convert to lowercase and strip spaces
      return text
    except Exception as e:
      print(e)
      return text

In [ ]:
def get_related_question(question):
    query = """
    CALL db.index.fulltext.queryNodes("questionIndex",$question)
    YIELD node, score
    RETURN node.name, score
    ORDER BY score DESC
    """
    result = graph.run(query, question=question).data()
    return result[0]

def get_related_data(action):
    query = """
    MATCH (a:Action {name: $action})
    OPTIONAL MATCH (a)-[:HAS_STEP]->(s:Step)
    OPTIONAL MATCH (a)-[:HAS_REQUIREMENT]->(r:Requirement)
    OPTIONAL MATCH (a)-[:HAS_KEY]->(k:Keys)
    WITH a, collect(DISTINCT s.name) AS steps, collect(DISTINCT r.name) AS requirements, collect(DISTINCT k.name) AS keys
    RETURN steps, requirements, keys
    """
    result = graph.run(query, action=action).data()
    return result[0]

def get_action(question):
    query = """
    MATCH (q:Question {name: $question})<-[:HAS_QUESTION]-(a:Action)
    RETURN a.name AS action
    """
    result = graph.run(query, question=question).data()
    return result[0]


def get_context(q_txt):
    related_data = get_related_question(q_txt)

    action=get_action(related_data['node.name'])

    related_data = get_related_data(action['action'])

    key_txt="#Key words:"
    keys=related_data['keys']
    for k in keys:
        key_txt=key_txt+k+','

    req_txt="#Prerequisites:"
    requirements=related_data['requirements']
    for r in requirements:
        req_txt=req_txt+r+','

    step_txt="#Steps to Follow:"
    steps=related_data['steps']
    for s in steps:
        step_txt=step_txt+s+','

    context=key_txt+'\n'+req_txt+'\n'+step_txt
    return context

In [ ]:
#Function that creates a prompt from instruction, context, category and response and tokenizes it
def collate_and_tokenize(examples):

    question=examples['0'][0]
    answer=examples['1'][0]
    q_text=preprocess_text(question)
    if q_text:
      context=get_context(q_text)
    else:
      context="No Context"

    #Merging into one prompt for tokenization and training
    prompt = f"""###System:Read the Context provided and answer the corresponding question.
    ###Context:
    {context}
    ###Question:
    {question}
    ###Answer:
    {answer}"""

    #Tokenize the prompt
    encoded = tokenizer(
        prompt,
        return_tensors="np",
        padding="max_length",
        truncation=True,
        ## Very critical to keep max_length at 1024.
        ## Anything more will lead to OOM on T4
        max_length=2048,
    )

    encoded["labels"] = encoded["input_ids"]
    return encoded

In [ ]:
#We will just keep the input_ids and labels that we add in function above.
columns_to_remove = ['0','1']

# tokenize the training and test datasets
tokenized_dataset_train = train_dataset.map(collate_and_tokenize,
                                            batched=True,
                                            batch_size=1,
                                            remove_columns=columns_to_remove)
tokenized_dataset_test = test_dataset.map(collate_and_tokenize,
                                          batched=True,
                                          batch_size=1,
                                          remove_columns=columns_to_remove)
#

Parameter 'function'=<function collate_and_tokenize at 0x7bf4a9188d30> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/899 [00:00<?, ? examples/s]

expected string or bytes-like object
expected string or bytes-like object


Map:   0%|          | 0/214 [00:00<?, ? examples/s]

expected string or bytes-like object
expected string or bytes-like object
expected string or bytes-like object


In [ ]:
#Check if tokenization looks good
input_ids = tokenized_dataset_train[2]['input_ids']

decoded = tokenizer.decode(input_ids, skip_special_tokens=True)
print(decoded)

###System:Read the Context and Chat History provided and answer the corresponding question.
    ###Context:
    #Key words:
#Prerequisites:Valid email address,Password,Agreement to terms and conditions,Agreement to privacy notice,First and last name,Payment information (credit card or debit card),Verification of phone number via SMS/text message,
#Steps to Follow:Create a valid email address,Get a valid phone number,Create a password,Agree to terms and conditions and our privacy notice,Fill in your first and last name,Phone number,Preferred language,Complete the signup process,Receive a text SMS to verify your phone number,Enter payment information (credit card or debit card),Confirm account registration via email,Use the app to request a ride,
    ###Question:
    Please help me create an uber account
    ###Answer:
    Creating an Uber account requires a valid email address and phone number. You'll also need to create a password and agree to terms and conditions and our privacy notic

In [ ]:
#Accelerate training models on larger batch sizes, we can use a fully sharded data parallel model.
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [ ]:
from peft import prepare_model_for_kbit_training

print_trainable_parameters(model)

#gradient checkpointing to save memory
model.gradient_checkpointing_enable()

# Freeze base model layers and cast layernorm in fp32
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)
print(model)

trainable params: 262364160 || all params: 1521392640 || trainable%: 17.24
PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiFlashAttention2(
          (q_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (dense): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
     

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
    'q_proj',
    'k_proj',
    'v_proj',
    'dense',
    'fc1',
    'fc2',
    ], #print(model) will show the modules to use
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

lora_model = get_peft_model(model, config)
print_trainable_parameters(lora_model)


lora_model = accelerator.prepare_model(lora_model)

trainable params: 23592960 || all params: 1544985600 || trainable%: 1.53


### Training the Model and saving to Hub
This is where, we setup the training arguments. These arguments have been carefully selected to improve memory utilization and also help increase performance. I played around with these for a while, before finalizing the following arguments.

Finally, I am saving the model weights , so we do not loose out work.

In [ ]:
%cd /content/drive/MyDrive/ontology/phi2/

/content/drive/MyDrive/ontology/phi2


In [ ]:
import time
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',  # Output directory for checkpoints and predictions
    overwrite_output_dir=True, # Overwrite the content of the output directory
    per_device_train_batch_size=2,  # Batch size for training
    per_device_eval_batch_size=2,  # Batch size for evaluation
    gradient_accumulation_steps=5, # number of steps before optimizing
    gradient_checkpointing=True,   # Enable gradient checkpointing
    gradient_checkpointing_kwargs={"use_reentrant": False},
    warmup_steps=50,  # Number of warmup steps
    #max_steps=1000,  # Total number of training steps
    num_train_epochs=2,  # Number of training epochs
    learning_rate=5e-5,  # Learning rate
    weight_decay=0.01,  # Weight decay
    optim="paged_adamw_8bit", #Keep the optimizer state and quantize it
    fp16=True, #Use mixed precision training
    #For logging and saving
    logging_dir='./logs',
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,  # Limit the total number of checkpoints
    evaluation_strategy="steps",
    eval_steps=100,
    load_best_model_at_end=True, # Load the best model at the end of training
)

trainer = Trainer(
    model=lora_model,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_test,
    args=training_args,
)

#Disable cache to prevent warning, renable for inference
model.config.use_cache = False

start_time = time.time()  # Record the start time
trainer.train()  # Start training
end_time = time.time()  # Record the end time

training_time = end_time - start_time  # Calculate total training time

print(f"Training completed in {training_time} seconds.")


The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.float16.


Step,Training Loss,Validation Loss
100,1.820100,0.485170
200,0.609600,0.471045
300,0.589800,0.466512
400,0.581600,0.465981
500,0.555800,0.467200
600,0.543400,0.469741
700,0.531600,0.475205
800,0.517300,0.478684
900,0.500800,0.480893
1000,0.490400,0.489742


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

Training completed in 11856.731600522995 seconds.


In [ ]:
#Save model to drive to ensure we save our work.
trainer.save_model("result/")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# Evaluate the fine-tuned model
results = trainer.evaluate()

# Print evaluation results
print(results)

{'eval_loss': 0.4659807085990906, 'eval_runtime': 19.1522, 'eval_samples_per_second': 5.221, 'eval_steps_per_second': 2.611, 'epoch': 2.0}


In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir logs

## Run Inference

**Note**: Ensure to stop your session and reconnect and reload the model before running the code below.

First we will run inference without the trained weights and check the output.

Next, lets run the model with lora config and check inference on it.

In [ ]:
from peft import PeftModel, PeftConfig

#Load the model weights from hub
model_id = "result/"
trained_model = PeftModel.from_pretrained(model, model_id)

#

## Calculate F1 Score

**Note**: Run all the cells to find average f1 score of test data set.

In [ ]:
import re, string
from collections import defaultdict, Counter

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

    #

In [ ]:
def spliit_and_return(txt):
  parts=txt.split("###Answer:")
  return parts[0],parts[1]

  #

In [ ]:
#Calculate Average L1 score for the trained model
from tqdm import tqdm

all_f1=[]
for i in tqdm (range (10), desc="Loading..."):
  input_ids = tokenized_dataset_test[i]['input_ids']
  decoded = tokenizer.decode(input_ids, skip_special_tokens=True)
  qustion_txt, ground_truth = spliit_and_return(decoded)
  # print("real",ground_truth)

  inputs = tokenizer(qustion_txt, return_tensors="pt",
                   return_attention_mask=False,
                   padding=True, truncation=True)
  inputs.to('cuda')
  outputs = trained_model.generate(**inputs, max_length=2048)
  out_put_text = tokenizer.batch_decode(outputs,skip_special_tokens=True)[0]
  qustion_txt, prediction = spliit_and_return(out_put_text)
  print("prediction",prediction)

  f1=f1_score(prediction, ground_truth)
  print(f1)
  all_f1.append(f1)

  #

Loading...:   0%|          | 0/10 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Loading...:  10%|█         | 1/10 [00:14<02:09, 14.37s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


prediction 
    Yes, you can.

To contact your driver, you can:

1. Tap the single tap icon on the top right corner of the screen.
2. Double tap the round phone icon under your driver's information.
3. Double tap the phone number or the 'Free Call' button.
4. Your phone will automatically call your driver according to the option you chose.
0.4814814814814815


Loading...:  20%|██        | 2/10 [00:31<02:06, 15.79s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


prediction 
    Yes, you can communicate with your driver using TalkBack.

To do this, you need to be matched with a driver. Once you are matched, you can single tap the round phone icon under your driver's information.

Next, double tap the phone number or the 'Free Call' button. Your phone will automatically call your driver according to the option you chose.
0.4121212121212121


Loading...:  30%|███       | 3/10 [00:42<01:34, 13.55s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

prediction 
    To request a ride with TalkBack, follow these steps:

1. Tap the “Where to” button.
2. Select your destination.
3. Tap the “Confirm pickup” button.
4. Wait for the driver to arrive.
5. When the driver arrives, tap the “Where to” button again.
6. Select your destination.
7. Tap the “Confirm pickup” button.
8. Wait for the driver to arrive.
9. When the driver arrives, tap the “Where to” button again.
10. Select your destination.
11. Tap the “Confirm pickup” button.
12. Wait for the driver to arrive.
13. When the driver arrives, tap the “Where to” button again.
14. Select your destination.
15. Tap the “Confirm pickup” button.
16. Wait for the driver to arrive.
17. When the driver arrives, tap the “Where to” button again.
18. Select your destination.
19. Tap the “Confirm pickup” button.
20. Wait for the driver to arrive.
21. When the driver arrives, tap the “Where to” button again.
22. Select your destination.
23. Tap the “Confirm pickup” button.
24. Wait for the driver to 

Loading...:  90%|█████████ | 9/10 [09:29<02:12, 132.83s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


prediction 
    To request an Uber ride using TalkBack, follow these steps:

1. Tap the “Where to” button.
2. Select your destination.
3. Tap the “Confirm pickup” button.
4. If you’re using a phone with a touch screen, double tap the “Where to” button.
5. If you’re using a phone with a physical keyboard, swipe through vehicle options.
6. Confirm your pickup location by tapping the “Confirm pickup” button.
7. If you’re using a phone with a touch screen, tap the “Confirm pickup” button.
8. If you’re using a phone with a physical keyboard, swipe through vehicle options.
9. Confirm your pickup location by tapping the “Confirm pickup” button.

If you’re using a phone with a touch screen, double tap the “Where to” button.

If you’re using a phone with a physical keyboard, swipe through vehicle options.

If you’re using a phone with a touch screen, tap the “Confirm pickup” button.

If you’re using a phone with a physical keyboard, swipe through vehicle options.

If you’re using a phone with a

Loading...: 100%|██████████| 10/10 [09:46<00:00, 58.65s/it]

prediction 
    When you request a ride with TalkBack, you'll see a map of your location.

1. Tap where to go.
2. Double tap to confirm your destination.
3. Swipe through vehicle options.
4. Confirm pickup button.
5. Your driver will arrive at your location.

If you're using TalkBack, you'll see a map of your location.

1. Tap where to go.
2. Double tap to confirm your destination.
3. Swipe through vehicle options.
4. Confirm pickup button.
5. Your driver will arrive at your location.
0.3529411764705882


In [ ]:
# average F1 score for test data set
avg_F1=sum(all_f1) / len(all_f1)
print("Average Score For Testing: ",avg_F1*100)

Average Score For Testing:  38.629035426499634
